<a href="https://colab.research.google.com/github/RashmikaReddy/Named-Entity-Recognition/blob/main/NER_BiLSTM%2BCRF_using_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from numpy import NaN

In [ ]:
!pip install nltk
import nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
!pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-x2o6owhp
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-x2o6owhp
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101076 sha256=8cb76eef67b610c6c5ddb5e5ddea5ca051c963033ab715ba33f0dea28e24aa05
  Stored in directory: /tmp/pip-ephem-wheel-cache-2w0dni58/wheels/67/d2/f4/96ae3c3c62d1e05abfc8860ad0c1207794726d44ebbbb547f3
Successfully built keras-contrib


In [ ]:
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from keras.layers import Input
from keras.models import Model
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite.metrics import flat_precision_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to /tmp/pip-install-4b1kdmtw/sklearn-crfsuite_58289f4e6df8476aab38a15183b11126
  Running command git clone --filter=blob:none --quiet https://github.com/MeMartijn/updated-sklearn-crfsuite.git /tmp/pip-install-4b1kdmtw/sklearn-crfsuite_58289f4e6df8476aab38a15183b11126
  Resolved https://github.com/MeMartijn/updated-sklearn-crfsuite.git to commit 675038761b4405f04691a83339d04903790e2b95
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.0 MB/s eta 0:00:00
  Created wheel for sklearn_crfsuite: filename=sklearn_crfsuite-0.3.6-py2.py3-none-any.whl size=10889 sha256=60ca10b985b93c28b7ef9e246036313d62ee58462163d93bfed4eb76051e7a59
  Stored in directory: /tmp/pip-ephem-wheel-cache-rm0ipsor/wheels/fb/c8/8a/95b4eccd3a273adbfb0a08d7f0e96d45d9f4aee82015d293c2
Successfully built sklear

In [ ]:
!pip install seqeval==0.0.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from keras.preprocessing.text import text_to_word_sequence
import pickle


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/EbayChallenge/Train_Tagged_Titles.tsv', sep="\t",on_bad_lines='skip')

In [ ]:
data.head(20)

,Record Number,Title,Token,Tag
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,Brand
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,NaN
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model
5,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Multi-color,Color
6,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,canvas,Fabric Type
7,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Multi-color,Color
8,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,canvas,Fabric Type
9,2,LOUIS VUITTON Petit Noe Drawstring Shoulder Ba...,LOUIS,Brand


In [ ]:
data['Tag'].unique()

array(['Brand', nan, 'MPN', 'Type', 'Model', 'Color', 'Fabric Type',
       'Closure', 'Pattern', 'Material', 'No Tag', 'Product Line',
       'Features', 'Theme', 'Country/Region of Manufacture', 'Occasion',
       'Department', 'Size', 'Handle/Strap Material', 'Handle Style',
       'Accents', 'Style', 'Hardware Material', 'Trim Material',
       'Measurement, dimension', 'Pocket Type', 'Character', 'Obscure',
       'Season', 'Character Family', 'Lining Material', 'Strap Drop',
       'Handle Drop'], dtype=object)

In [ ]:
#Number of texts we have are 4997
data['Record Number'].nunique()

4997

In [ ]:
data_df = data[['Record Number','Token','Tag']]

In [ ]:
data_df = data_df.rename(columns={"Record Number": "sentence_id", "Token": "words","Tag":"labels"})


In [ ]:
data_df

,sentence_id,words,labels
0,1,LOUIS,Brand
1,1,VUITTON,NaN
2,1,M40096,MPN
3,1,Handbag,Type
4,1,Priscilla,Model
...,...,...,...
55075,5000,Top,No Tag
55076,5000,Closure,No Tag
55077,5000,Retail,No Tag
55078,5000,$,No Tag


In [ ]:
data.head(5)

,Record Number,Title,Token,Tag
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,Brand
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,NaN
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model


In [ ]:
# Using insert(), add empty column at first position
data_df.insert(3,"POS Tagging", " ")
print(data_df)


       sentence_id      words  labels POS Tagging
0                1      LOUIS   Brand            
1                1    VUITTON     NaN            
2                1     M40096     MPN            
3                1    Handbag    Type            
4                1  Priscilla   Model            
...            ...        ...     ...         ...
55075         5000        Top  No Tag            
55076         5000    Closure  No Tag            
55077         5000     Retail  No Tag            
55078         5000          $  No Tag            
55079         5000        595  No Tag            

[55080 rows x 4 columns]


In [ ]:
dictionary = {}
input
unique_tags = set()
prev_rownumber = 1
s = ""
prev_title = ""
for index,row in data.iterrows():
  title = row['Title']
  rownumber = row['Record Number']
  if rownumber != prev_rownumber:
    if len(s) != 0:
      dictionary[prev_title] = s[1:]
      s = ""
      prev_rownumber = rownumber
  if rownumber == prev_rownumber:
    if row['Tag'] == 'No Tag':
      s = s+";"+"O"
      data_df['labels'][index] = 'O'
      unique_tags.add("O")
    elif row['Tag'] is not NaN:
      s = s+";"+"B-"+row['Tag']
      data_df['labels'][index] = "B-"+row['Tag']
      unique_tags.add("B-"+row['Tag'])
      prev_tag = row['Tag']
    elif row['Tag'] is NaN:
      s = s+";"+"I-"+prev_tag
      data_df['labels'][index] = "I-"+prev_tag
      unique_tags.add("I-"+prev_tag)
    prev_rownumber = rownumber
    prev_title = title
dictionary[prev_title] = (s.lstrip()).rstrip()

In [ ]:
print(data_df)


       sentence_id      words   labels POS Tagging
0                1      LOUIS  B-Brand            
1                1    VUITTON  I-Brand            
2                1     M40096    B-MPN            
3                1    Handbag   B-Type            
4                1  Priscilla  B-Model            
...            ...        ...      ...         ...
55075         5000        Top        O            
55076         5000    Closure        O            
55077         5000     Retail        O            
55078         5000          $        O            
55079         5000        595        O            

[55080 rows x 4 columns]


In [ ]:
for index,row in data_df.iterrows():
  word = [row['words']]
  data_df['POS Tagging'][index] = nltk.tag.pos_tag(word)[0][1]


In [ ]:
print(data_df)


       sentence_id      words   labels POS Tagging
0                1      LOUIS  B-Brand         NNP
1                1    VUITTON  I-Brand          NN
2                1     M40096    B-MPN          NN
3                1    Handbag   B-Type          NN
4                1  Priscilla  B-Model          NN
...            ...        ...      ...         ...
55075         5000        Top        O          JJ
55076         5000    Closure        O          NN
55077         5000     Retail        O          JJ
55078         5000          $        O           $
55079         5000        595        O          CD

[55080 rows x 4 columns]


In [ ]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['words'].values.tolist(),
                                                       s['POS Tagging'].values.tolist(),
                                                       s['labels'].values.tolist())]
        self.grouped = self.df.groupby("sentence_id").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [ ]:
#Displaying one full sentence
getter = sentence(data_df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas'

In [ ]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

None


In [ ]:
sentences = getter.sentences


In [ ]:
print(sentences)

[[('LOUIS', 'NNP', 'B-Brand'), ('VUITTON', 'NN', 'I-Brand'), ('M40096', 'NN', 'B-MPN'), ('Handbag', 'NN', 'B-Type'), ('Priscilla', 'NN', 'B-Model'), ('Multi-color', 'NN', 'B-Color'), ('canvas', 'NN', 'B-Fabric Type'), ('Multi-color', 'NN', 'B-Color'), ('canvas', 'NN', 'B-Fabric Type')], [('LOUIS', 'NNP', 'B-Brand'), ('VUITTON', 'NN', 'I-Brand'), ('Petit', 'NN', 'B-Model'), ('Noe', 'NN', 'I-Model'), ('Drawstring', 'VBG', 'B-Closure'), ('Shoulder', 'NN', 'B-Type'), ('Bag', 'NN', 'B-Type'), ('Monogram', 'NN', 'B-Pattern'), ('Leather', 'NN', 'B-Material'), ('M42226', 'NN', 'B-MPN'), ('39SD442', 'CD', 'O')], [('LOUIS', 'NNP', 'B-Brand'), ('VUITTON', 'NN', 'I-Brand'), ('Damier', 'NN', 'B-Product Line'), ('Azur', 'NN', 'B-Color'), ('Pochette', 'NN', 'B-Product Line'), ('Bosphore', 'IN', 'B-Model'), ('Shoulder', 'NN', 'B-Type'), ('Bag', 'NN', 'B-Type'), ('N51112', 'NN', 'B-MPN'), ('LV', 'NN', 'B-Brand'), ('Auth', 'NN', 'O'), ('yt523', 'NN', 'O')], [('GUCCI', 'NN', 'B-Brand'), ('Bamboo', 'NN', 

In [ ]:
# Number of data points passed in each iteration
batch_size = 64 
# Passes through entire dataset
epochs = 8
# Maximum length of review
max_len = 75 
# Dimension of embedding vector
embedding = 40 

Preprocessing Data
We will process our text data before feeding to the network.

Here word_to_index dictionary used to convert word into index value and tag_to_index is for the labels. So overall we represent each word as integer.


In [ ]:
#Getting unique words and labels from data
words = list(data_df['words'].unique())
tags = list(data_df['labels'].unique())
# Dictionary word:index pair
# word is key and its value is corresponding index

word_to_index = {w : i + 2 for i, w in enumerate(words)}
word_to_index["PAD"] = 0
word_to_index["UNK"] = 1

# Dictionary label:index pair
# label is key and value is index.
tag_to_index = {t : i + 1 for i, t in enumerate(tags)}
tag_to_index["PAD"] = 0

idx2word = {i: w for w, i in word_to_index.items()}
idx2tag = {i: w for w, i in tag_to_index.items()}

In [ ]:
tag_to_index = list(tag_to_index.items())

In [ ]:
tag_to_index[-2]

('I-Strap Drop', 60)

In [ ]:
print(word_to_index)

{'LOUIS': 2, 'VUITTON': 3, 'M40096': 4, 'Handbag': 5, 'Priscilla': 6, 'Multi-color': 7, 'canvas': 8, 'Petit': 9, 'Noe': 10, 'Drawstring': 11, 'Shoulder': 12, 'Bag': 13, 'Monogram': 14, 'Leather': 15, 'M42226': 16, '39SD442': 17, 'Damier': 18, 'Azur': 19, 'Pochette': 20, 'Bosphore': 21, 'N51112': 22, 'LV': 23, 'Auth': 24, 'yt523': 25, 'GUCCI': 26, 'Bamboo': 27, '2Way': 28, 'Brown': 29, 'fm1002': 30, 'Rank': 31, 'AB': 32, 'Vintage': 33, 'Gucci': 34, 'Sherry': 35, 'line': 36, 'PVC': 37, 'Clutch': 38, 'bag': 39, 'From': 40, 'Japan': 41, 'A128': 42, '1970s': 43, 'NYC': 44, 'Bonnie': 45, 'Cashin': 46, 'Coach': 47, 'Gray': 48, 'Saddle': 49, 'Pouch': 50, 'Crossbody': 51, 'Epi': 52, 'Serviette': 53, 'Fermoir': 54, 'Business': 55, 'M54358': 56, 'gt2071': 57, 'Womens': 58, 'Chanel': 59, 'Leo': 60, 'Lion': 61, 'Flap': 62, 'Chevron': 63, 'Lambskin': 64, 'Medium': 65, 'NEW': 66, 'FOSSIL': 67, 'GREEN': 68, 'PLAID': 69, 'CANVAS': 70, 'ESTATE': 71, 'CVS': 72, 'BRIEFCASE': 73, ',': 74, 'CROSS': 75, 'BOD

In [ ]:
print(idx2word)

{2: 'LOUIS', 3: 'VUITTON', 4: 'M40096', 5: 'Handbag', 6: 'Priscilla', 7: 'Multi-color', 8: 'canvas', 9: 'Petit', 10: 'Noe', 11: 'Drawstring', 12: 'Shoulder', 13: 'Bag', 14: 'Monogram', 15: 'Leather', 16: 'M42226', 17: '39SD442', 18: 'Damier', 19: 'Azur', 20: 'Pochette', 21: 'Bosphore', 22: 'N51112', 23: 'LV', 24: 'Auth', 25: 'yt523', 26: 'GUCCI', 27: 'Bamboo', 28: '2Way', 29: 'Brown', 30: 'fm1002', 31: 'Rank', 32: 'AB', 33: 'Vintage', 34: 'Gucci', 35: 'Sherry', 36: 'line', 37: 'PVC', 38: 'Clutch', 39: 'bag', 40: 'From', 41: 'Japan', 42: 'A128', 43: '1970s', 44: 'NYC', 45: 'Bonnie', 46: 'Cashin', 47: 'Coach', 48: 'Gray', 49: 'Saddle', 50: 'Pouch', 51: 'Crossbody', 52: 'Epi', 53: 'Serviette', 54: 'Fermoir', 55: 'Business', 56: 'M54358', 57: 'gt2071', 58: 'Womens', 59: 'Chanel', 60: 'Leo', 61: 'Lion', 62: 'Flap', 63: 'Chevron', 64: 'Lambskin', 65: 'Medium', 66: 'NEW', 67: 'FOSSIL', 68: 'GREEN', 69: 'PLAID', 70: 'CANVAS', 71: 'ESTATE', 72: 'CVS', 73: 'BRIEFCASE', 74: ',', 75: 'CROSS', 76: 

In [ ]:
# Converting each sentence into list of index from list of tokens
X = [[word_to_index[w[0]] for w in s] for s in sentences]

# Padding each sequence to have same length  of each word
X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index["PAD"])

In [ ]:
print(tag_to_index)

[('B-Brand', 1), ('I-Brand', 2), ('B-MPN', 3), ('B-Type', 4), ('B-Model', 5), ('B-Color', 6), ('B-Fabric Type', 7), ('I-Model', 8), ('B-Closure', 9), ('B-Pattern', 10), ('B-Material', 11), ('O', 12), ('B-Product Line', 13), ('B-Features', 14), ('I-Product Line', 15), ('I-Material', 16), ('B-Theme', 17), ('B-Country/Region of Manufacture', 18), ('B-Occasion', 19), ('B-Department', 20), ('B-Size', 21), ('I-Type', 22), ('I-Closure', 23), ('B-Handle/Strap Material', 24), ('I-Department', 25), ('B-Handle Style', 26), ('B-Accents', 27), ('B-Style', 28), ('B-Hardware Material', 29), ('I-Hardware Material', 30), ('I-Handle Style', 31), ('I-Fabric Type', 32), ('I-Pattern', 33), ('I-Theme', 34), ('B-Trim Material', 35), ('I-Features', 36), ('I-Style', 37), ('I-Accents', 38), ('B-Measurement, dimension', 39), ('I-Measurement, dimension', 40), ('B-Pocket Type', 41), ('I-Pocket Type', 42), ('I-MPN', 43), ('B-Character', 44), ('I-Character', 45), ('B-Obscure', 46), ('I-Occasion', 47), ('I-Handle/Str

In [ ]:
for s in sentences:
  for w in s:
    print(w[2])
    print(type(w[2]))
    print(tag_to_index[w[2]])

B-Brand
<class 'str'>


TypeError: ignored

In [ ]:
tag_to_index = dict(tag_to_index)


In [ ]:
print(type(tag_to_index))

<class 'dict'>


In [ ]:
# Convert label to index
y = [[tag_to_index[w[2]] for w in s] for s in sentences]

# padding
y = pad_sequences(maxlen = max_len, sequences = y, padding = "post", value = tag_to_index["PAD"])

In [ ]:
print(X[0:5])

[[ 2  3  4  5  6  7  8  7  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 2  3  9 10 11 12 13 14 15 16 17  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 2  3 18 19 20 21 12 13 22 23 24 25  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [26 27 28 12 13 15 29 24 30  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [31 32 33 34 35 36 37 15 38 39 29 40 41 42  0  0  0  0  0  0  0

In [ ]:
print(y[0:5])

[[ 1  2  3  4  5  6  7  6  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 1  2  5  8  9  4  4 10 11  3 12  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 1  2 13  6 13  5  4  4  3  1 12 12  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 1 13 14  4  4 11  6 12  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [12 12 12  1 13 15 11 16  4  4  6 12 12 12  0  0  0  0  0  0  0

In [ ]:
num_tag = data_df['labels'].nunique()
# One hot encoded labels
y = [to_categorical(i, num_classes = num_tag + 1) for i in y]

In [ ]:
print(y[0:5])

[array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


In [ ]:
print("Size of training input data : ", X_train.shape)
print("Size of training output data : ", np.array(y_train).shape)
print("Size of testing input data : ", X_test.shape)
print("Size of testing output data : ", np.array(y_test).shape)


Size of training input data :  (3997, 75)
Size of training output data :  (3997, 75, 61)
Size of testing input data :  (1000, 75)
Size of testing output data :  (1000, 75, 61)


In [ ]:
# Let's check the first sentence before and after processing.
print('*****Before Processing first sentence : *****\n', ' '.join([w[0] for w in sentences[0]]))
print('*****After Processing first sentence : *****\n ', X[0])

*****Before Processing first sentence : *****
 LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas
*****After Processing first sentence : *****
  [2 3 4 5 6 7 8 7 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]


In [ ]:
# First label before and after processing.
print('*****Before Processing first sentence : *****\n', ' '.join([w[2] for w in sentences[0]]))
print('*****After Processing first sentence : *****\n ', y[0])

*****Before Processing first sentence : *****
 B-Brand I-Brand B-MPN B-Type B-Model B-Color B-Fabric Type B-Color B-Fabric Type
*****After Processing first sentence : *****
  [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
num_tags = data_df['labels'].nunique()

# Model definition
input = Input(shape=(max_len,))
model = Embedding(input_dim=len(words)+2, output_dim=embedding, # n_words + 2 (PAD & UNK)
                  input_length=max_len, mask_zero=True)(input)  # default: 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(num_tags+1)  # CRF layer, n_tags+1(PAD)
out = crf(model)  # output
model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])
model.summary()

AttributeError: ignored

Bidirectional LSTM-CRF network

In [ ]:
# Model architecture
input = Input(shape = (max_len,))
model = Embedding(input_dim = len(words) + 2, output_dim = embedding, input_length = max_len, mask_zero = True)(input)
model = Bidirectional(LSTM(units = 50, return_sequences=True, recurrent_dropout=0.1))(model)
model = TimeDistributed(Dense(50, activation="relu"))(model)
crf = CRF(num_tags+1)  # CRF layer
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()


AttributeError: ignored

**Evaluating the model performance.**

We will use precision, recall and f1-score metrics to evaluate the performance of the model since the accuracy is not a good metric for this dataset because we have an unequal number of data points in each class.

In [ ]:
f1_score = flat_f1_score(y_test, y_pred, average = 'macro')
print(f1_score)

0.5891590907214499


In [ ]:
print(y_test)

[['B-Brand', 'I-Brand', 'B-Model', 'B-Pattern', 'B-Material', 'B-Features', 'B-Size', 'O', 'B-Type', 'B-Type', 'B-Type'], ['O', 'B-Brand', 'I-Brand', 'B-Pattern', 'B-Model', 'I-Model', 'B-Type', 'B-Type', 'B-MPN', 'O', 'B-Brand', 'O'], ['O', 'B-Brand', 'B-Model', 'O', 'O', 'B-Pattern', 'B-Color', 'O', 'B-Color', 'B-Material'], ['O', 'B-Brand', 'I-Brand', 'B-Type'], ['O', 'O', 'O', 'O', 'B-Brand', 'I-Brand', 'B-Color', 'B-Pattern', 'B-Type'], ['B-Brand', 'B-Accents', 'B-Model', 'B-Type', 'B-Type', 'B-Material', 'B-Color', 'O', 'O', 'B-Country/Region of Manufacture', 'B-MPN'], ['B-Brand', 'B-Product Line', 'B-Color', 'O', 'B-Color', 'B-Material', 'B-Closure', 'B-Type', 'O', 'O', 'B-Type', 'O', 'O'], ['B-Brand', 'B-Size', 'B-Type', 'B-Material', 'B-Type', 'I-Type', 'B-Accents', 'B-MPN'], ['B-Brand', 'I-Brand', 'B-Model', 'B-Accents', 'B-Material', 'B-Type', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-Brand', 'B-Type', 'B-Type', 'I-Type', 'B-Type', 'B-Brand', 'B-Fabric Type', 'B-Material', 'B-Color

In [ ]:
print(y_pred)

[['B-Brand', 'I-Brand', 'B-Model', 'B-Pattern', 'B-Material', 'B-Features', 'B-Size', 'O', 'B-Type', 'B-Type', 'B-Type'], ['O', 'B-Brand', 'I-Brand', 'B-Pattern', 'B-Model', 'I-Model', 'B-Type', 'B-Type', 'B-MPN', 'O', 'B-Brand', 'O'], ['O', 'B-Brand', 'B-Model', 'I-Model', 'O', 'B-Pattern', 'B-Color', 'O', 'B-Color', 'B-Material'], ['O', 'B-Brand', 'I-Brand', 'B-Type'], ['O', 'O', 'O', 'O', 'B-Brand', 'I-Brand', 'B-Color', 'B-Pattern', 'B-Type'], ['B-Brand', 'B-Accents', 'B-Model', 'B-Type', 'B-Type', 'B-Material', 'B-Color', 'O', 'O', 'O', 'O'], ['B-Brand', 'B-Model', 'B-Color', 'O', 'B-Color', 'B-Material', 'B-Closure', 'B-Type', 'O', 'O', 'B-Type', 'O', 'O'], ['B-Brand', 'B-Size', 'B-Type', 'B-Material', 'B-Type', 'I-Type', 'B-Accents', 'O'], ['B-Brand', 'I-Brand', 'B-Model', 'B-Accents', 'B-Material', 'B-Type', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-Brand', 'B-Type', 'B-Type', 'I-Type', 'B-Type', 'B-Brand', 'B-Fabric Type', 'B-Material', 'B-Color', 'B-Color', 'O'], ['O', 'B-Brand', 'B

In [ ]:
report = flat_classification_report(y_test,y_pred)
print(report)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  labels : list, optional
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  labels : list, optional


                                 precision    recall  f1-score   support

                      B-Accents       0.79      0.71      0.75       190
                        B-Brand       0.94      0.94      0.94       914
                    B-Character       1.00      0.18      0.31        11
             B-Character Family       1.00      0.20      0.33         5
                      B-Closure       0.87      0.89      0.88        61
                        B-Color       0.92      0.93      0.93       876
B-Country/Region of Manufacture       0.83      0.74      0.79        47
                   B-Department       0.96      0.94      0.95       176
                  B-Fabric Type       0.95      0.92      0.93       122
                     B-Features       0.79      0.68      0.73       119
                  B-Handle Drop       0.00      0.00      0.00         3
                 B-Handle Style       0.88      0.86      0.87        70
        B-Handle/Strap Material       0.95      0.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  labels : list, optional


Defining the parameters for LSTM network

In [ ]:
# Number of data points passed in each iteration
batch_size = 64 
# Passes through entire dataset
epochs = 8
# Maximum length of review
max_len = 75 
# Dimension of embedding vector
embedding = 40 